**Carregar um classificador**

In [1]:
import pickle 
import numpy as np
import os

Carregar a base novamente por motivos de aprendizado:

In [2]:
with open('credit.pkl', 'rb') as f:
    X_credit_train, y_credit_train, X_credit_test, y_credit_test = pickle.load(f)

X_credit = np.concatenate([X_credit_train, X_credit_test])
y_credit = np.concatenate([y_credit_train, y_credit_test])

X_credit.shape, y_credit.shape

((2000, 3), (2000,))

In [3]:
rede_neural = pickle.load(open('rede_neural_final.sav', 'rb'))
arvore = pickle.load(open('arvore_final.sav', 'rb'))
svm = pickle.load(open('support_vector_final.sav', 'rb'))

In [4]:
novo_registro = X_credit[0]
novo_registro, novo_registro.shape

(array([-1.3754462 ,  0.50631087,  0.10980934]), (3,))

O array está em 1D. o método `predict` exige 2D e para convertes usamos o método do numpy `.reshape(-1,1)`

In [5]:
### Previsao com a rede neural
novo_registro = novo_registro.reshape(1, -1) # Uma linha e Três colunas
print(f'Rede Neural {rede_neural.predict(novo_registro)}') 
print(f'Arvore {arvore.predict(novo_registro)}')
print(f'SVM {svm.predict(novo_registro)}')

Rede Neural [0]
Arvore [0]
SVM [0]


# Combinação de Classificadores

In [6]:
paga = 0
nao_paga = 1 

resposta_rede = rede_neural.predict(novo_registro)
resposta_arv = arvore.predict(novo_registro)
resposta_svm = svm.predict(novo_registro)

resposta_rede, resposta_arv, resposta_svm

(array([0], dtype=int64), array([0], dtype=int64), array([0], dtype=int64))

In [7]:
## 
# Rede neural
if resposta_rede[0] == 1:
    nao_paga += 1
else:
    paga += 1
# arvore
if resposta_arv[0] == 1:
    nao_paga += 1
else:
    paga += 1
# SVM 
if resposta_svm[0] == 1:
    nao_paga += 1
else:
    paga += 1    

### Mensagem
if paga > nao_paga:
    print("Cliente pagará o empréstimo")
elif paga == nao_paga:
    print(f'empate')
else:
    print("cliente não pagará o empréstimo")

Cliente pagará o empréstimo


## Rejeição  

Para trabalhar com a rejeição precisamos trabalhar com *probabilidade* e o _scikit learn_ tem isso _built-in_ em todos os seus classificadores.  

**Nota**: O SVM treinado anteriormente não tinha nos parâmetros o `propabilitity = True`, que é `False` por _default_, sendo necessário portanto, editá-lo para ter. 

In [8]:
prob_rede_neural = rede_neural.predict_proba(novo_registro)
prob_rede_neural

array([[1.00000000e+00, 3.83755356e-16]])

In [9]:
confianca_rede_neural = prob_rede_neural.max()
confianca_rede_neural
# 99% de chance de ser um registro que paga o empréstimo

0.9999999999999997

In [10]:
prob_arvore = arvore.predict_proba(novo_registro)
confianca_arvore = prob_arvore.max()
confianca_arvore

1.0

In [13]:
prob_svm = svm.predict_proba(novo_registro)
confianca_svm = prob_svm.max()
confianca_svm

0.999997556232365

In [17]:
## Setando a confiança mínima como um pouco maior que svm
confianca_minima = 0.9999999
# contagem dos algoritmos utilizados
algoritmos = 0 

# Rede neural
if confianca_rede_neural > confianca_minima:
    algoritmos += 1
    if resposta_rede[0] == 1:
        nao_paga += 1
    else:
        paga += 1
# arvore
if confianca_arvore > confianca_minima:
    algoritmos += 1
    if resposta_arv[0] == 1:
        nao_paga += 1
    else:
        paga += 1

# SVM 
if confianca_svm > confianca_minima:
    algoritmos += 1
    if resposta_svm[0] == 1:
        nao_paga += 1
    else:
        paga += 1   

### Mensagem
if paga > nao_paga:
    print("Cliente pagará o empréstimo baseado em {}".format(algoritmos))
elif paga == nao_paga:
    print(f'empate')
else:
    print("cliente não pagará o empréstimo baseados em {} algoritmos".format(algoritmos))

Cliente pagará o empréstimo baseado em 2
